## Assignment02: Data Cleaning

In [1]:
import pandas as pd

In [2]:
# Load the dataset
df = pd.read_csv('2004.csv')
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2004,1,12,1,623.0,630,901.0,915,UA,462,...,7,11,0,NaN,0,0,0,0,0,0
1,2004,1,13,2,621.0,630,911.0,915,UA,462,...,16,16,0,NaN,0,0,0,0,0,0
2,2004,1,14,3,633.0,630,920.0,915,UA,462,...,4,15,0,NaN,0,0,0,0,0,0
3,2004,1,15,4,627.0,630,859.0,915,UA,462,...,4,10,0,NaN,0,0,0,0,0,0
4,2004,1,16,5,635.0,630,918.0,915,UA,462,...,3,13,0,NaN,0,0,0,0,0,0


In [4]:
df.shape

(7129270, 29)

In [3]:
# Check for missing values
df.isnull().sum()

Year                       0
Month                      0
DayofMonth                 0
DayOfWeek                  0
DepTime               127757
CRSDepTime                 0
ArrTime               141541
CRSArrTime                 0
UniqueCarrier              0
FlightNum                  0
TailNum                  127
ActualElapsedTime     141541
CRSElapsedTime             0
AirTime               141541
ArrDelay              141541
DepDelay              127757
Origin                     0
Dest                       0
Distance                   0
TaxiIn                     0
TaxiOut                    0
Cancelled                  0
CancellationCode     7001506
Diverted                   0
CarrierDelay               0
WeatherDelay               0
NASDelay                   0
SecurityDelay              0
LateAircraftDelay          0
dtype: int64

In [7]:
print(df['Cancelled'].value_counts())
print(df['Diverted'].value_counts())

Cancelled
0    7001513
1     127757
Name: count, dtype: int64
Diverted
0    7115486
1      13784
Name: count, dtype: int64


**Since our analysis would look into two different categories: Delayed Flights and Cancelled Flights, we would then extract dataframes that would be beneficial for each analysis. This would also ensure we are not uploading a large size of dataset to Hive to perform analysis.**

### 1. Delayed Flight Patterns Analysis 

We should exclude cancelled/diverted flights to ensure we're analyzing only completed flights with valid performance data. Meaning, we don't need to look into flights that never flew at all (cancelled flights) or flights that have landed elsewhere (diverted flights).

In [5]:
# Drop cancelled and diverted flights
df_delay = df[(df['Cancelled'] == 0) & (df['Diverted'] == 0)]
df_delay.shape

(6987729, 29)

In [6]:
# Missing values in df_delay
df_delay.isnull().sum()

Year                       0
Month                      0
DayofMonth                 0
DayOfWeek                  0
DepTime                    0
CRSDepTime                 0
ArrTime                    0
CRSArrTime                 0
UniqueCarrier              0
FlightNum                  0
TailNum                    0
ActualElapsedTime          0
CRSElapsedTime             0
AirTime                    0
ArrDelay                   0
DepDelay                   0
Origin                     0
Dest                       0
Distance                   0
TaxiIn                     0
TaxiOut                    0
Cancelled                  0
CancellationCode     6987722
Diverted                   0
CarrierDelay               0
WeatherDelay               0
NASDelay                   0
SecurityDelay              0
LateAircraftDelay          0
dtype: int64

Since we dont' have cancellation code data, we can drop that column along with Cancelled and Diverted columns

In [11]:
df_delay.drop(columns=['CancellationCode','Cancelled','Diverted'], inplace=True)

C:\Users\BRTHASHW\AppData\Local\Temp\ipykernel_33392\3511590207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_delay.drop(columns=['CancellationCode','Cancelled','Diverted'], inplace=True)


In [12]:
df_delay.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2004,1,12,1,623.0,630,901.0,915,UA,462,...,ORD,CLT,599,7,11,0,0,0,0,0
1,2004,1,13,2,621.0,630,911.0,915,UA,462,...,ORD,CLT,599,16,16,0,0,0,0,0
2,2004,1,14,3,633.0,630,920.0,915,UA,462,...,ORD,CLT,599,4,15,0,0,0,0,0
3,2004,1,15,4,627.0,630,859.0,915,UA,462,...,ORD,CLT,599,4,10,0,0,0,0,0
4,2004,1,16,5,635.0,630,918.0,915,UA,462,...,ORD,CLT,599,3,13,0,0,0,0,0


This df_delay would be uploaded to Hive to perform analysis on Delay Patterns & Factor. Hence, we would now rename the columns for standardization purposes when we upload to Hive.

In [13]:
# Rename columns to lower case and snake_case
df_delay.columns = [
    'year', 'month', 'day_of_month', 'day_of_week', 'dep_time', 'crs_dep_time',
    'arr_time', 'crs_arr_time', 'unique_carrier', 'flight_num', 'tail_num',
    'actual_elapsed_time', 'crs_elapsed_time', 'air_time', 'arr_delay',
    'dep_delay', 'origin', 'dest', 'distance', 'taxi_in', 'taxi_out','carrier_delay',
    'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay'
]

In [14]:
# Save file to upload to Hive 
df_delay.to_csv("df_delay_2004.csv", index=False, header=True)

### 2. Cancellation Analysis

To analyse cancelled flights, we would extract rows of cancelled flight data only. 

In [15]:
df_cancelled = df[df['Cancelled'] == 1]
df_cancelled.shape

(127757, 29)

In [16]:
# Check for missing values
df_cancelled.isnull().sum()

Year                      0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime              127757
CRSDepTime                0
ArrTime              127757
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                 127
ActualElapsedTime    127757
CRSElapsedTime            0
AirTime              127757
ArrDelay             127757
DepDelay             127757
Origin                    0
Dest                      0
Distance                  0
TaxiIn                    0
TaxiOut                   0
Cancelled                 0
CancellationCode          0
Diverted                  0
CarrierDelay              0
WeatherDelay              0
NASDelay                  0
SecurityDelay             0
LateAircraftDelay         0
dtype: int64

For df_cancelled, delayed flight information is not relevant. Hence, we can proceed to dropping the columns related to delayed flights.

In [18]:
df_cancelled.drop(columns=['DepTime','ArrTime','ActualElapsedTime','AirTime','ArrDelay','DepDelay',
                          'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay'], inplace=True)

C:\Users\BRTHASHW\AppData\Local\Temp\ipykernel_33392\2177776573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelled.drop(columns=['DepTime','ArrTime','ActualElapsedTime','AirTime','ArrDelay','DepDelay',


In [19]:
df_cancelled.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted
12,2004,1,26,1,630,915,UA,462,000000,105,ORD,CLT,599,0,0,1,B,0
21,2004,1,4,7,1505,1725,UA,463,000000,260,ORD,PDX,1739,0,0,1,C,0
23,2004,1,6,2,1505,1725,UA,463,000000,260,ORD,PDX,1739,0,0,1,B,0
24,2004,1,7,3,1510,1732,UA,463,000000,262,ORD,PDX,1739,0,0,1,C,0
25,2004,1,8,4,1510,1732,UA,463,000000,262,ORD,PDX,1739,0,0,1,B,0


In [20]:
df_cancelled['Diverted'].value_counts()

Diverted
0    127757
Name: count, dtype: int64

In [21]:
# Drop 'Diverted' column as well
df_cancelled.drop(columns=['Diverted'], inplace=True)
df_cancelled.head()

C:\Users\BRTHASHW\AppData\Local\Temp\ipykernel_33392\2335958726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelled.drop(columns=['Diverted'], inplace=True)


,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode
12,2004,1,26,1,630,915,UA,462,000000,105,ORD,CLT,599,0,0,1,B
21,2004,1,4,7,1505,1725,UA,463,000000,260,ORD,PDX,1739,0,0,1,C
23,2004,1,6,2,1505,1725,UA,463,000000,260,ORD,PDX,1739,0,0,1,B
24,2004,1,7,3,1510,1732,UA,463,000000,262,ORD,PDX,1739,0,0,1,C
25,2004,1,8,4,1510,1732,UA,463,000000,262,ORD,PDX,1739,0,0,1,B


Same with df_delay, we would rename the columns in df_cancelled and save the file to be uploaded to Hive for analysis

In [22]:
# Rename columns to lower case and snake_case
df_cancelled.columns = [
    'year', 'month', 'day_of_month', 'day_of_week', 'crs_dep_time',
    'crs_arr_time', 'unique_carrier', 'flight_num', 'tail_num',
    'crs_elapsed_time', 'origin', 'dest', 'distance', 'taxi_in', 'taxi_out',
    'cancelled', 'cancellation_code']

In [23]:
# Save file to upload to Hive 
df_cancelled.to_csv("df_cancelled_2004.csv", index=False, header=True)